In [47]:
%load_ext autoreload
%autoreload 2
import sys
import pandas as pd
sys.path.append("..")
from Services import GeoService, MapFeatureAggregator, AddressToCoordinateConverter

import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [14]:
myrealty = pd.read_csv("../data/myrealty_apartments.csv")

def format_address(address):
    s = address
    splits = s.split(", ")
    split_count = len(splits)
    if split_count >= 3:
        s = splits[-1]
    return s


myrealty['location'] = myrealty['location'].apply(format_address)
converter = AddressToCoordinateConverter("streets.csv")
myrealty["coordinates"] = myrealty["location"].apply(converter.convert)

In [15]:
significant_places = { 
    "opera" : (40.18600,44.51509),
    "malibu" : (40.18306,44.50799),
    "aua" : (40.19243,44.50446),
    "zvartnoc" : (40.1523,44.4005)
    # avan
    # cemeteries
    # try 20-30 locations
}
geo_service = GeoService(
    significant_places,
    radius = 300
)
featurer = MapFeatureAggregator(geo_service)
significant_distances = featurer.significant_distances(myrealty, "coordinates")
amenities = featurer.amenities_count(myrealty, "coordinates")

In [37]:
def feature_importance(data, y_col):
    X = data.drop(y_col, axis=1)
    y = data[y_col]
    rf = RandomForestRegressor(n_estimators=100) 
    rf.fit(X, y)
    feature_importances = pd.DataFrame({
        'feature': X.columns,
        'importance': rf.feature_importances_
    })
    feature_importances = feature_importances.sort_values(
        by = 'importance',
        ascending = False
    )
    return feature_importances

In [54]:
df = myrealty[["price", "coordinates"]]
df["latitude"] = df["coordinates"].apply(lambda x: x[0])
df["longitude"] = df["coordinates"].apply(lambda x: x[1])
df = df.drop(columns = ["coordinates"])
df = pd.concat([df, significant_distances], axis = 1)
feature_importance(df, "price").head(10)

,feature,importance
4,aua,0.278446
2,opera,0.223376
0,latitude,0.151661
5,zvartnoc,0.122752
1,longitude,0.119377
3,malibu,0.104388


In [55]:
X = df.drop("price", axis=1)
y = df["price"]
rf = RandomForestRegressor(n_estimators=100) 
rf.fit(X, y)
rf.score(X, y)

0.8106942792097667

In [61]:
mean_squared_error(rf.predict(X), y)

3952382851.5134835